## Example

### Definicion de funciones necesarias para generacion malla

In [60]:
%config IPCompleter.greedy=True
import numpy as np
import os
import bempp.api
from tools.mesh_converter import *
from tools.force_calculation import *
from tools.apbs_tools import *
import griddata

### Seleccion de proteina a simular, contenidas en carpetas pqr_files

Se extraen las cargas, la ubicacion de estas y se realiza el mallado a traves de Nanoshaper

In [61]:
grid, q, x_q = pqrtomesh(directory='C:\\Users\\ian\Desktop\\forces_calculation',protein='arg',forcefield='amber',density=1.6,probe_radius=1.4,build_mesh='no')
grid2, q2, x_q2 = pqrtomesh(directory='C:\\Users\\ian\Desktop\\forces_calculation',protein='sphere',forcefield='symmetric_t30',density=12.0,probe_radius=1.4,build_mesh='yes')
bempp.api.PLOT_BACKEND = "gmsh" #gmsh o paraview
grid.plot()
grid2.plot()

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [62]:
# Function to calculate the potential by charges at boundary
ep_in = 2.
ep_ex = 80.
k = 0.125

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)
dirichl_space2 = bempp.api.function_space(grid2, "DP", 0)
neumann_space2 = bempp.api.function_space(grid2, "DP", 0)

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    suma = 0
    for k in range(len(q)):
        suma = suma + q[k]/(np.linalg.norm(x-x_q[k]))
    result[:] = suma/(4*np.pi*ep_in)
    
@bempp.api.real_callable
def charges_fun2(x, n, domain_index, result):
    global q2, x_q2, ep_in
    suma = 0
    for k in range(len(q2)):
        suma = suma + q2[k]/(np.linalg.norm(x-x_q2[k]))
    result[:] = suma/(4*np.pi*ep_in)
    
charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)
charged_grid_fun2 = bempp.api.GridFunction(dirichl_space2, fun=charges_fun2)

rhs = np.concatenate([charged_grid_fun.coefficients, np.zeros(neumann_space.global_dof_count),  
                      np.zeros(neumann_space2.global_dof_count),charged_grid_fun2.coefficients])


C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [63]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
#Poisson equation operators
identity1 = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in11   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in11   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_in22   = laplace.single_layer(neumann_space2, dirichl_space2, dirichl_space2)
dlp_in22   = laplace.double_layer(dirichl_space2, dirichl_space2, dirichl_space2)

#Poisson-Boltzmann equations operators
identity2 = sparse.identity(dirichl_space2, dirichl_space2, dirichl_space2)
slp_out11  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
slp_out12  = modified_helmholtz.single_layer(neumann_space, dirichl_space2, dirichl_space2, k)
slp_out21  = modified_helmholtz.single_layer(neumann_space2, dirichl_space, dirichl_space, k)
slp_out22  = modified_helmholtz.single_layer(neumann_space2, dirichl_space2, dirichl_space2, k)
dlp_out11  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)
dlp_out12  = modified_helmholtz.double_layer(dirichl_space, dirichl_space2, dirichl_space2, k)
dlp_out21  = modified_helmholtz.double_layer(dirichl_space2, dirichl_space, dirichl_space, k)
dlp_out22  = modified_helmholtz.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(4, 4)
blocked[0, 0] = 0.5*identity1 + dlp_in11
blocked[0, 1] = -slp_in11
blocked[1, 0] = 0.5*identity1 - dlp_out11
blocked[1, 1] = (ep_in/ep_ex)*slp_out11
blocked[1, 2] = -dlp_out21
blocked[1, 3] = (ep_in/ep_ex)*slp_out21
blocked[2, 0] = - dlp_out12
blocked[2, 1] = (ep_in/ep_ex)*slp_out12
blocked[2, 2] = 0.5*identity2-dlp_out22
blocked[2, 3] = (ep_in/ep_ex)*slp_out22
blocked[3, 2] = 0.5*identity2 + dlp_in22
blocked[3, 3] = -slp_in22


op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [64]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-3, maxiter=1000, restart = 2000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 55 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [65]:
a = dirichl_space.global_dof_count
a1 = neumann_space.global_dof_count
b = dirichl_space2.global_dof_count
b1 = neumann_space2.global_dof_count

solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:a])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[a:(a+a1)])
                                          
solution_dirichl2 = bempp.api.GridFunction(dirichl_space2, 
                                          coefficients=x[(a+a1):(a+a1+b)])
solution_neumann2 = bempp.api.GridFunction(neumann_space2, 
                                          coefficients=x[(a+a1+b):])

In [66]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

h=0.001
F_qf,_ = fixedcharge_forces(x_q,q,h,neumann_space,dirichl_space,solution_neumann,solution_dirichl)
f_db,f_ib = boundary_forces(solution_neumann,solution_dirichl,grid,k,ep_ex,ep_in)
  
print("Total solvation force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(F_qf[0]+f_db[0]+f_ib[0],F_qf[1]+f_db[1]+f_ib[1],F_qf[2]+f_db[2]+f_ib[2]))
print("Total reaction force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(F_qf[0],F_qf[1],F_qf[2]))
print("Total dielectric boundary force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_db[0],f_db[1],f_db[2]))
print("Total ionic boundary force: {:5.3e} {:5.3e} {:5.3e} [kJ/molA]".format(f_ib[0],f_ib[1],f_ib[2]))
# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 4.184*2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total solvation force:    -0.7939   -0.4658    0.5576 [kJ/molA]
Total reaction force:    48.0233  -11.4962   21.4754 [kJ/molA]
Total dielectric boundary force:   -48.7366   11.0451  -20.8874 [kJ/molA]
Total ionic boundary force: -8.065e-02 -1.477e-02 -3.039e-02 [kJ/molA]
Total solvation energy: -206.39 [kcal/Mol]


In [67]:
slp_q2 = bempp.api.operators.potential.laplace.single_layer(neumann_space2, x_q2.transpose())
dlp_q2 = bempp.api.operators.potential.laplace.double_layer(dirichl_space2, x_q2.transpose())
phi_q2 = slp_q2*solution_neumann2 - dlp_q2*solution_dirichl2

h=0.001
F_qf2, FFe = fixedcharge_forces(x_q2,q2,h,neumann_space2,dirichl_space2,solution_neumann2,solution_dirichl2)
f_db2,f_ib2 = boundary_forces(solution_neumann2,solution_dirichl2,grid2,k,ep_ex,ep_in)

print("Total solvation force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(F_qf2[0]+f_db2[0]+f_ib2[0],F_qf2[1]+f_db2[1]+f_ib2[1],F_qf2[2]+f_db2[2]+f_ib2[2]))
print("Total reaction force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(F_qf2[0],F_qf2[1],F_qf2[2]))
print("Total dielectric boundary force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_db2[0],f_db2[1],f_db2[2]))
print("Total ionic boundary force: {:5.3e} {:5.3e} {:5.3e} [kJ/molA]".format(f_ib2[0],f_ib2[1],f_ib2[2]))
# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 4.184*2*np.pi*332.064*np.sum(q2*phi_q2).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total solvation force:     0.6299   -0.0086   -0.3594 [kJ/molA]
Total reaction force:     0.8092    0.1810   -0.5872 [kJ/molA]
Total dielectric boundary force:    -0.1843   -0.1895    0.2319 [kJ/molA]
Total ionic boundary force: 5.039e-03 -4.812e-05 -4.092e-03 [kJ/molA]
Total solvation energy: -1142.65 [kcal/Mol]


## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.

[4] Sergio Decherchi and Walter Rocchia. A general and Robust Ray-Casting-Based
Algorithm for Triangulating Surfaces at the Nanoscale. PLOS ONE, 8(4):1–15, 2013.

[5] Timo Betcke and Matthew W. Scroggs. Bempp-cl: A fast python based just-in-time
compiling boundary element library. Journal of Open Source Software, 6(59):2879,
2021.

[6] S. Decherchi, A. Spitaleri, J. Stone and W. Rocchia, "NanoShaper-VMD interface: computing and visualizing surfaces, pockets and channels in
molecular systems". Bioinformatics, in press, 2018.

In [68]:
grid.plot()
grid2.plot()

In [69]:
FFe

array([[-1.06172185e+02,  1.06049891e-01, -2.62394906e-01],
       [ 1.06981419e+02,  7.49389191e-02, -3.24841577e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])